In [ ]:
import os
import sys
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define and set the working directory
new_directory = '/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/02 - Esagoni'
if os.path.isdir(new_directory):
    os.chdir(new_directory)
    sys.path.append(new_directory)
    print("Current directory:", os.getcwd())
else:
    print("Directory does not exist:", new_directory)

Mounted at /content/drive
Current directory: /content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/02 - Esagoni


# Load and Display Merged Dataset from Excel File

In [ ]:
import pandas as pd

# Load the Excel file
df_brescia = pd.read_excel('/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/00 - Pulizia dati/dataset_unito.xlsx')

# Display the first rows to check that everything is fine
print("Merged Dataset:")
print(df_brescia.head())

Merged Dataset:
        Comune  Autovetture  Autobus e filobus    Autocarri    Motrici    \
0  Acquafredda          996                    0          185          0   
1         Adro         4812                    1          699         19   
2     Agnosine         1242                    0          228          5   
3   Alfianello         1585                    0          221          7   
4         Anfo          307                    0           41          0   

   Rimorchi    Motocicli    Motocarri    Altri veicoli    \
0           7          156            8                0   
1          54          797           36                0   
2           8          264           17                0   
3          16          218            7                0   
4           0           59            9                0   

   Autovetture circolanti Euro 0    ...  \
0                               47  ...   
1                              237  ...   
2                              104  .

In [ ]:
!pip install h3==3.* geopandas shapely matplotlib osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 9.7 MB/s eta 0:00:00


In [ ]:
import osmnx as ox
import folium
import shapely
import h3
import geopandas as gpd
import random

In [ ]:
# Renames the 'Pop_tot' column to 'Population' for better clarity:
df_brescia = df_brescia.rename(columns={"Pop_tot": "Population"})

---

## Spatial Mapping of Municipalities in Brescia
This code creates a spatial visualization of the municipalities in the province of Brescia, Italy, by generating a hexagonal grid (using H3 resolution) and performing a spatial join between hexagons and municipalities. It then visualizes the results on a map, with each hexagon colored randomly based on the municipality it belongs to. Additionally, the centroids of the municipalities are marked on the map for reference.

In [ ]:
# Assuming df_brescia exists and has the 'Comune' column
comuni_brescia = df_brescia['Comune'].tolist()

# Step 1: Base map centered on Brescia
m = folium.Map(location=[45.54, 10.22], zoom_start=10)

# Step 2: Get the geometry for the entire province of Brescia
provincia_gdf = ox.geocode_to_gdf("Brescia, Italy")
brescia_polygon = provincia_gdf.geometry.iloc[0]

# Step 3: Generate H3 grid
resolution = 8
h3_indices = h3.polyfill_geojson(brescia_polygon.__geo_interface__, resolution)
hex_geometries = [
    shapely.geometry.Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in h3_indices
]
hexagon_df = gpd.GeoDataFrame({
    "h3_id": list(h3_indices),
    "geometry": hex_geometries
}, crs="EPSG:4326")

# Step 4: Download the polygons of the municipalities from OSMnx using only the names in df_brescia
comuni_poligoni = []
for comune in comuni_brescia:
    try:
        gdf = ox.geocode_to_gdf(f"{comune}, Brescia, Italy")
        gdf["COMUNE"] = comune
        comuni_poligoni.append(gdf)
    except Exception as e:
        print(f"Error in municipality {comune}: {e}")

# Combine all municipalities into a single GeoDataFrame
gdf_comuni = pd.concat(comuni_poligoni, ignore_index=True)
gdf_comuni = gdf_comuni.set_geometry("geometry").to_crs("EPSG:4326")

# Step 5: Spatial join between hexagons and municipalities (correct!)
joined = gpd.sjoin(hexagon_df, gdf_comuni[['COMUNE', 'geometry']], predicate='intersects', how='left')

# Resolve the issue: take the first municipality for each hexagon
comune_per_esagono = joined.groupby('h3_id')['COMUNE'].first().reset_index()

# Merge with hexagon_df
hexagon_df = hexagon_df.merge(comune_per_esagono, on='h3_id', how='left')

# Step 6: Random coloring for each municipality
colors = {
    comune: f"#{random.randint(0, 0xFFFFFF):06x}" for comune in hexagon_df['COMUNE'].dropna().unique()
}

# Debug: print assignment information
print("Hexagons assigned to municipalities:", hexagon_df['COMUNE'].notna().sum())
print("Hexagons not assigned:", hexagon_df['COMUNE'].isna().sum())
print("Unique municipalities found:", sorted(hexagon_df['COMUNE'].dropna().unique()))

# 🔎 Verification of the two specific municipalities
for comune_check in ["Brione", "Cividate Camuno"]:
    if comune_check in hexagon_df['COMUNE'].values:
        print(f"The municipality {comune_check} is present in the hexagons.")
    else:
        print(f"The municipality {comune_check} is NOT present in the hexagons.")

# Step 7: Add hexagons to the map
for _, row in hexagon_df.iterrows():
    color = colors.get(row['COMUNE'], "#808080")
    folium.Polygon(
        locations=[(y, x) for x, y in row["geometry"].exterior.coords],
        fill=True,
        fill_opacity=0.6,
        color=color,
        weight=0.5,
        popup=f"Municipality: {row['COMUNE'] or 'Not assigned'}<br>H3: {row['h3_id']}"
    ).add_to(m)

# Step 8: Add the centroids of the municipalities
for _, row in gdf_comuni.iterrows():
    folium.CircleMarker(
        location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
        radius=5,
        color='red',
        fill=True,
        fill_color='red',
        popup=f"Municipality: {row['COMUNE']}"
    ).add_to(m)

# Display the map
m

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
m.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/02 - Esagoni/hexagons_with_centroids.html")

## Quality Checks on Geographic and Administrative Data
Performs quality control by verifying the validity of municipal polygons and their coordinate reference system (CRS), and ensures consistency between expected and assigned municipalities across datasets.

In [ ]:
# --- Geometric check: polygon validity
valid = gdf_comuni.is_valid.sum()
invalid = len(gdf_comuni) - valid
print(f"Valid polygons: {valid}")
print(f"Invalid polygons: {invalid}")

# --- Geometric check: coordinate reference system
print("Polygons' CRS:", gdf_comuni.crs)

# --- Logical check: match between hexagon_df and df_brescia municipalities
found_municipalities = set(hexagon_df['COMUNE'].dropna().unique())
expected_municipalities = set(df_brescia['Comune'].unique())

missing_municipalities = expected_municipalities - found_municipalities
extra_municipalities = found_municipalities - expected_municipalities

print(f"Assigned municipalities (from hexagons): {len(found_municipalities)}")
print(f"Expected municipalities (from df_brescia): {len(expected_municipalities)}")
print(f"Missing municipalities: {len(missing_municipalities)} → {sorted(missing_municipalities)}")
print(f"Extra municipalities (not expected): {len(extra_municipalities)} → {sorted(extra_municipalities)}")

Valid polygons: 205
Invalid polygons: 0
Polygons' CRS: epsg:4326
Assigned municipalities (from hexagons): 205
Expected municipalities (from df_brescia): 205
Missing municipalities: 0 → []
Extra municipalities (not expected): 0 → []


# Load Current and 2030 Forecast Data (Conservative and Optimistic Scenarios)


In [ ]:
# Load current data
df_comuni = pd.read_csv('/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/01 - Stime + Previsioni/df_attuale.csv')

# Load conservative 2030 forecast data
df_2030_conservativo = pd.read_csv('/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/01 - Stime + Previsioni/df_2030_conservativo.csv')

# Load optimistic 2030 forecast data
df_2030_ottimistico = pd.read_csv('/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/01 - Stime + Previsioni/df_2030_ottimistico.csv')

## Standardization and Validation of Municipality Names
Renames the municipality column in hexagon_df, ensures consistent formatting of names across all datasets by converting to lowercase and removing extra spaces, and checks for missing values in the "Comune" column.

In [ ]:
# Rename column "COMUNE" to "Comune"
hexagon_df = hexagon_df.rename(columns={"COMUNE": "Comune"})

# Standardize the "Comune" column (not the index)
hexagon_df['Comune'] = hexagon_df['Comune'].astype(str).str.strip().str.lower()
df_comuni['Comune'] = df_comuni['Comune'].astype(str).str.strip().str.lower()
df_2030_conservativo['Comune'] = df_2030_conservativo['Comune'].astype(str).str.strip().str.lower()
df_2030_ottimistico['Comune'] = df_2030_ottimistico['Comune'].astype(str).str.strip().str.lower()

# Check for NaN values in the municipality names
print(f"NaN values in hexagon_df['Comune']: {hexagon_df['Comune'].isnull().sum()}")
print(f"NaN values in df_comuni['Comune']: {df_comuni['Comune'].isnull().sum()}")
print(f"NaN values in df_2030_conservativo['Comune']: {df_2030_conservativo['Comune'].isnull().sum()}")
print(f"NaN values in df_2030_ottimistico['Comune']: {df_2030_ottimistico['Comune'].isnull().sum()}")

NaN values in hexagon_df['Comune']: 0
NaN values in df_comuni['Comune']: 0
NaN values in df_2030_conservativo['Comune']: 0
NaN values in df_2030_ottimistico['Comune']: 0


## Municipality Matching Across Datasets
Checks for mismatches in municipality names between hexagon_df and the other datasets (df_comuni, df_2030_conservativo, df_2030_ottimistico) to identify missing or unmatched records.

In [ ]:
# Comparison between hexagon_df and df_comuni
# Check if there are municipalities in df_comuni that are not in hexagon_df
comuni_non_in_hexagon = df_comuni[~df_comuni['Comune'].isin(hexagon_df['Comune'])]
print("Municipalities in df_comuni but not in hexagon_df:")
print(comuni_non_in_hexagon)

# Check if there are municipalities in hexagon_df that are not in df_comuni
comuni_non_in_df_comuni = hexagon_df[~hexagon_df['Comune'].isin(df_comuni['Comune'])]
print("Municipalities in hexagon_df but not in df_comuni:")
print(comuni_non_in_df_comuni)

# Comparison between hexagon_df and df_2030_conservativo
# Check if there are municipalities in hexagon_df that are not in df_2030_conservativo
comuni_non_in_conservativo = hexagon_df[~hexagon_df['Comune'].isin(df_2030_conservativo['Comune'])]
print("\nMunicipalities in hexagon_df but not in df_2030_conservativo:")
print(comuni_non_in_conservativo)

# Check if there are municipalities in df_2030_conservativo that are not in hexagon_df
comuni_non_in_hexagon_conservativo = df_2030_conservativo[~df_2030_conservativo['Comune'].isin(hexagon_df['Comune'])]
print("\nMunicipalities in df_2030_conservativo but not in hexagon_df:")
print(comuni_non_in_hexagon_conservativo)

# Comparison between hexagon_df and df_2030_ottimistico
# Check if there are municipalities in hexagon_df that are not in df_2030_ottimistico
comuni_non_in_ottimistico = hexagon_df[~hexagon_df['Comune'].isin(df_2030_ottimistico['Comune'])]
print("\nMunicipalities in hexagon_df but not in df_2030_ottimistico:")
print(comuni_non_in_ottimistico)

# Check if there are municipalities in df_2030_ottimistico that are not in hexagon_df
comuni_non_in_hexagon_ottimistico = df_2030_ottimistico[~df_2030_ottimistico['Comune'].isin(hexagon_df['Comune'])]
print("\nMunicipalities in df_2030_ottimistico but not in hexagon_df:")
print(comuni_non_in_hexagon_ottimistico)

Municipalities in df_comuni but not in hexagon_df:
Empty DataFrame
Columns: [Comune, Pop_tot, Quota Popolazione, EV Stimati (Attuali), Domanda Mensile (kWh) - Attuali, Domanda Annua (kWh) - Attuali, Domanda Giornaliera (kWh) - Attuali, Colonnine Necessarie - Attuali]
Index: []
Municipalities in hexagon_df but not in df_comuni:
Empty GeoDataFrame
Columns: [h3_id, geometry, Comune]
Index: []

Municipalities in hexagon_df but not in df_2030_conservativo:
Empty GeoDataFrame
Columns: [h3_id, geometry, Comune]
Index: []

Municipalities in df_2030_conservativo but not in hexagon_df:
Empty DataFrame
Columns: [Comune, Pop_Comune_2030_Conservativo, Quota Popolazione 2030 Conservativo, EV Stimati (2030 - Conservativo), Domanda Mensile (kWh) - Conservativo, Domanda Annua (kWh) - Conservativo, Domanda Giornaliera (kWh) - Conservativo, Colonnine Necessarie - Conservativo]
Index: []

Municipalities in hexagon_df but not in df_2030_ottimistico:
Empty GeoDataFrame
Columns: [h3_id, geometry, Comune]
Ind

---

## Join and Aggregate Estimates by Municipality and Hexagon

### Current Scenario
Data validation involves counting hexagons per municipality, standardizing municipality names across datasets, and calculating statistics per hexagon. It also checks for discrepancies between municipalities in different datasets (hexagon_df and df_comuni), ensuring consistency and identifying any missing or extra municipalities."

In [ ]:
# === CURRENT SCENARIO ===

# --- 1. Count how many hexagons each municipality has in your hexagon_df
esagoni_per_comune = hexagon_df.groupby("Comune").size().rename("n_esagoni")

# --- 2. Standardize the municipality names to avoid issues
# Attention: always work on the "Comune" column, NEVER on the indices!
hexagon_df["Comune"] = hexagon_df["Comune"].astype(str).str.strip().str.lower()
df_comuni["Comune"] = df_comuni["Comune"].astype(str).str.strip().str.lower()

# --- 3. Add n_esagoni to df_comuni using a merge
df_comuni = df_comuni.merge(
    esagoni_per_comune.rename("n_esagoni").reset_index().rename(columns={"Comune": "Comune_nome_match"}),
    left_on="Comune",
    right_on="Comune_nome_match",
    how="left"
)

# --- 4. Replace NaN with 0 (for municipalities without hexagons)
df_comuni["n_esagoni"] = df_comuni["n_esagoni"].fillna(0)

# --- 5. Calculate the per hexagon share, avoiding division by zero
cols = [
    "EV Stimati (Attuali)",
    "Domanda Giornaliera (kWh) - Attuali",
    "Domanda Mensile (kWh) - Attuali",
    "Domanda Annua (kWh) - Attuali",
    "Colonnine Necessarie - Attuali"
]

for col in cols:
    df_comuni[f"{col} per esagono"] = df_comuni.apply(
        lambda row: row[col] / row["n_esagoni"] if row["n_esagoni"] > 0 else 0,
        axis=1
    )

# --- 6. Merge with hexagon_df to assign per hexagon estimates
hexagon_df_att = hexagon_df.merge(
    df_comuni[[f"{col} per esagono" for col in cols] + ["Comune"]],
    on="Comune",
    how="left"
)

# --- 7. Automatic checks on municipalities

# Municipalities in df_comuni without hexagons (n_esagoni == 0)
comuni_senza_esagoni = df_comuni[df_comuni["n_esagoni"] == 0]
print(f"Municipalities without hexagons ({len(comuni_senza_esagoni)}):")
print(comuni_senza_esagoni["Comune"].tolist())

# Municipalities in hexagon_df not found in df_comuni
comuni_hex = set(hexagon_df["Comune"].unique())
comuni_df = set(df_comuni["Comune"].unique())
comuni_non_in_df = comuni_hex - comuni_df
print(f"Municipalities in hexagon_df not found in df_comuni ({len(comuni_non_in_df)}):")
print(list(comuni_non_in_df))

# Municipalities in df_comuni not found in hexagon_df
comuni_non_in_hex = comuni_df - comuni_hex
print(f"Municipalities in df_comuni not found in hexagon_df ({len(comuni_non_in_hex)}):")
print(list(comuni_non_in_hex))

Municipalities without hexagons (0):
[]
Municipalities in hexagon_df not found in df_comuni (0):
[]
Municipalities in df_comuni not found in hexagon_df (0):
[]


#### Visualization of Hexagonal Data with Heatmap Color Coding

In [ ]:
import branca.colormap as cm

# Select the column to display (you can change here)
colonna_heatmap = "Domanda Giornaliera (kWh) - Attuali per esagono"

# Create the map centered on Brescia
mappa = folium.Map(location=[45.54, 10.22], zoom_start=10)

# Create a continuous color map between min and max
min_val = hexagon_df_att[colonna_heatmap].min()
max_val = hexagon_df_att[colonna_heatmap].max()
colormap = cm.linear.YlOrRd_09.scale(min_val, max_val)
colormap.caption = f"{colonna_heatmap}"

# Add the hexagons with color based on the variable
for _, row in hexagon_df_att.iterrows():
    valore = row[colonna_heatmap]
    colore = colormap(valore) if not pd.isna(valore) else "#cccccc"
    folium.Polygon(
        locations=[(y, x) for x, y in row["geometry"].exterior.coords],
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=colore,
        weight=0.2,
        popup=f"{row['Comune']}<br>{colonna_heatmap}: {round(valore, 2)}"
    ).add_to(mappa)

# Add the legend
colormap.add_to(mappa)

# Display the map
mappa

Output hidden; open in https://colab.research.google.com to view.

#### Logarithmic Heatmap Visualization with Detailed Tooltips

In [ ]:
import numpy as np
import branca.colormap as cm
import folium

# Select the column to display for coloring
colonna_heatmap = "Domanda Giornaliera (kWh) - Attuali per esagono"

# Create the map centered on Brescia
mappa = folium.Map(location=[45.54, 10.22], zoom_start=10)

# Calculate the logarithmic values with offset to avoid log(0)
offset = 1e-6
valori_originali = hexagon_df_att[colonna_heatmap]
valori_log = np.log10(valori_originali + offset)

# Create logarithmic colormap
min_log = valori_log.min()
max_log = valori_log.max()
colormap = cm.linear.YlOrRd_09.scale(min_log, max_log)
colormap.caption = f"{colonna_heatmap} (log scale)"

# Add the hexagons with detailed tooltip
for _, row in hexagon_df_att.iterrows():
    valore = row[colonna_heatmap]
    if pd.isna(valore):
        colore = "#cccccc"
    else:
        valore_log = np.log10(valore + offset)
        colore = colormap(valore_log)

    # Create rich tooltip text
    comune = row['Comune']
    tooltip_text = f"<b>Comune:</b> {comune.title()}<br>"
    for col in [
        "EV Stimati (Attuali) per esagono",
        "Domanda Giornaliera (kWh) - Attuali per esagono",
        "Domanda Mensile (kWh) - Attuali per esagono",
        "Domanda Annua (kWh) - Attuali per esagono",
        "Colonnine Necessarie - Attuali per esagono"
    ]:
        valore_col = row[col]
        valore_mostrato = round(valore_col, 2) if not pd.isna(valore_col) else "N/A"
        tooltip_text += f"<b>{col}:</b> {valore_mostrato}<br>"

    tooltip = folium.Tooltip(tooltip_text, sticky=True)

    folium.Polygon(
        locations=[(y, x) for x, y in row["geometry"].exterior.coords],
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=colore,
        weight=0.2,
        tooltip=tooltip
    ).add_to(mappa)

# Add the legend
colormap.add_to(mappa)

# Display the map
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/02 - Esagoni/log_heatmap_current_scenario.html")

### Projection to 2030 - Conservative Scenario

In [ ]:
# === SCENARIO 2030: CONSERVATIVE ===

# --- 1. Count how many hexagons each municipality has in your hexagon_df
esagoni_per_comune = hexagon_df.groupby("Comune").size().rename("n_esagoni")

# --- 2. Standardize municipality names to avoid issues
hexagon_df["Comune"] = hexagon_df["Comune"].astype(str).str.strip().str.lower()
df_2030_conservativo["Comune"] = df_2030_conservativo["Comune"].astype(str).str.strip().str.lower()

# --- 3. Add n_esagoni to df_2030_conservativo using a merge
df_2030_conservativo = df_2030_conservativo.merge(
    esagoni_per_comune.rename("n_esagoni").reset_index().rename(columns={"Comune": "Comune_nome_match"}),
    left_on="Comune",
    right_on="Comune_nome_match",
    how="left"
)

# --- 4. Replace NaN with 0 (municipalities without hexagons)
df_2030_conservativo["n_esagoni"] = df_2030_conservativo["n_esagoni"].fillna(0)

# --- 5. Calculate the per-hexagon share, avoiding division by zero
cols = [
    "EV Stimati (2030 - Conservativo)",
    "Domanda Giornaliera (kWh) - Conservativo",
    "Domanda Mensile (kWh) - Conservativo",
    "Domanda Annua (kWh) - Conservativo",
    "Colonnine Necessarie - Conservativo"
]

for col in cols:
    df_2030_conservativo[f"{col} per esagono"] = df_2030_conservativo.apply(
        lambda row: row[col] / row["n_esagoni"] if row["n_esagoni"] > 0 else 0,
        axis=1
    )

# --- 6. Merge with hexagon_df to assign per-hexagon estimates
hexagon_df_cons = hexagon_df.merge(
    df_2030_conservativo[[f"{col} per esagono" for col in cols] + ["Comune"]],
    on="Comune",
    how="left"
)

# --- 7. Automatic check for municipalities

# Municipalities in df_2030_conservativo without hexagons (n_esagoni == 0)
comuni_senza_esagoni = df_2030_conservativo[df_2030_conservativo["n_esagoni"] == 0]
print(f"Municipalities without hexagons ({len(comuni_senza_esagoni)}):")
print(comuni_senza_esagoni["Comune"].tolist())

# Municipalities in hexagon_df not present in df_2030_conservativo
comuni_hex = set(hexagon_df["Comune"].unique())
comuni_df = set(df_2030_conservativo["Comune"].unique())
comuni_non_in_df = comuni_hex - comuni_df
print(f"Municipalities in hexagon_df not found in df_2030_conservativo ({len(comuni_non_in_df)}):")
print(list(comuni_non_in_df))

# Municipalities in df_2030_conservativo not present in hexagon_df
comuni_non_in_hex = comuni_df - comuni_hex
print(f"Municipalities in df_2030_conservativo not found in hexagon_df ({len(comuni_non_in_hex)}):")
print(list(comuni_non_in_hex))

Municipalities without hexagons (0):
[]
Municipalities in hexagon_df not found in df_2030_conservativo (0):
[]
Municipalities in df_2030_conservativo not found in hexagon_df (0):
[]


#### Visualization of Hexagonal Data with Heatmap Color Coding -  Conservative Scenario

In [ ]:
import branca.colormap as cm

# Select the column to display (you can change it here)
colonna_heatmap = "Domanda Giornaliera (kWh) - Conservativo per esagono"

# Create the map centered on Brescia
mappa = folium.Map(location=[45.54, 10.22], zoom_start=10)

# Create a continuous color map between min and max
min_val = hexagon_df_cons[colonna_heatmap].min()
max_val = hexagon_df_cons[colonna_heatmap].max()
colormap = cm.linear.YlOrRd_09.scale(min_val, max_val)
colormap.caption = f"{colonna_heatmap}"

# Add the hexagons with color based on the variable
for _, row in hexagon_df_cons.iterrows():
    valore = row[colonna_heatmap]
    colore = colormap(valore) if not pd.isna(valore) else "#cccccc"
    folium.Polygon(
        locations=[(y, x) for x, y in row["geometry"].exterior.coords],
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=colore,
        weight=0.2,
        popup=f"{row['Comune']}<br>{colonna_heatmap}: {round(valore, 2)}"
    ).add_to(mappa)

# Add the legend
colormap.add_to(mappa)

# Display the map
mappa

Output hidden; open in https://colab.research.google.com to view.

#### Logarithmic Heatmap Visualization with Detailed Tooltips - Conservative Scenario

In [ ]:
import numpy as np
import branca.colormap as cm
import folium

# Select the column to display for coloring
colonna_heatmap = "Domanda Giornaliera (kWh) - Conservativo per esagono"

# Create the map centered on Brescia
mappa = folium.Map(location=[45.54, 10.22], zoom_start=10)

# Calculate logarithmic values with offset to avoid log(0)
offset = 1e-6
valori_originali = hexagon_df_cons[colonna_heatmap]
valori_log = np.log10(valori_originali + offset)

# Create logarithmic colormap
min_log = valori_log.min()
max_log = valori_log.max()
colormap = cm.linear.YlOrRd_09.scale(min_log, max_log)
colormap.caption = f"{colonna_heatmap} (log scale)"

# Add hexagons with detailed tooltips
for _, row in hexagon_df_cons.iterrows():
    valore = row[colonna_heatmap]
    if pd.isna(valore):
        colore = "#cccccc"
    else:
        valore_log = np.log10(valore + offset)
        colore = colormap(valore_log)

    # Create rich tooltip text
    comune = row['Comune']
    tooltip_text = f"<b>Comune:</b> {comune.title()}<br>"
    for col in [
        "EV Stimati (2030 - Conservativo) per esagono",
        "Domanda Giornaliera (kWh) - Conservativo per esagono",
        "Domanda Mensile (kWh) - Conservativo per esagono",
        "Domanda Annua (kWh) - Conservativo per esagono",
        "Colonnine Necessarie - Conservativo per esagono"
    ]:
        valore_col = row[col]
        valore_mostrato = round(valore_col, 2) if not pd.isna(valore_col) else "N/A"
        tooltip_text += f"<b>{col}:</b> {valore_mostrato}<br>"

    tooltip = folium.Tooltip(tooltip_text, sticky=True)

    folium.Polygon(
        locations=[(y, x) for x, y in row["geometry"].exterior.coords],
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=colore,
        weight=0.2,
        tooltip=tooltip
    ).add_to(mappa)

# Add the legend
colormap.add_to(mappa)

# Display the map
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/02 - Esagoni/log_heatmap_conservative_scenario.html")

### Projection to 2030 - Optimistic Scenario

In [ ]:
# === SCENARIO 2030: OPTIMISTIC ===

# --- 1. Count how many hexagons each municipality has in your hexagon_df
esagoni_per_comune = hexagon_df.groupby("Comune").size().rename("n_esagoni")

# --- 2. Standardize municipality names to avoid issues
hexagon_df["Comune"] = hexagon_df["Comune"].astype(str).str.strip().str.lower()
df_2030_ottimistico["Comune"] = df_2030_ottimistico["Comune"].astype(str).str.strip().str.lower()

# --- 3. Add n_esagoni to df_2030_ottimistico using a merge
df_2030_ottimistico = df_2030_ottimistico.merge(
    esagoni_per_comune.rename("n_esagoni").reset_index().rename(columns={"Comune": "Comune_nome_match"}),
    left_on="Comune",
    right_on="Comune_nome_match",
    how="left"
)

# --- 4. Replace NaN with 0 (municipalities without hexagons)
df_2030_ottimistico["n_esagoni"] = df_2030_ottimistico["n_esagoni"].fillna(0)

# --- 5. Calculate the per-hexagon share, avoiding division by zero
cols = [
    "EV Stimati (2030 - Ottimistico)",
    "Domanda Giornaliera (kWh) - Ottimistico",
    "Domanda Mensile (kWh) - Ottimistico",
    "Domanda Annua (kWh) - Ottimistico",
    "Colonnine Necessarie - Ottimistico"
]

for col in cols:
    df_2030_ottimistico[f"{col} per esagono"] = df_2030_ottimistico.apply(
        lambda row: row[col] / row["n_esagoni"] if row["n_esagoni"] > 0 else 0,
        axis=1
    )

# --- 6. Merge with hexagon_df to assign per-hexagon estimates
hexagon_df_ott = hexagon_df.merge(
    df_2030_ottimistico[[f"{col} per esagono" for col in cols] + ["Comune"]],
    on="Comune",
    how="left"
)

# --- 7. Automatic check for municipalities

# Municipalities in df_2030_ottimistico without hexagons (n_esagoni == 0)
comuni_senza_esagoni = df_2030_ottimistico[df_2030_ottimistico["n_esagoni"] == 0]
print(f"Municipalities without hexagons ({len(comuni_senza_esagoni)}):")
print(comuni_senza_esagoni["Comune"].tolist())

# Municipalities in hexagon_df not present in df_2030_ottimistico
comuni_hex = set(hexagon_df["Comune"].unique())
comuni_df = set(df_2030_ottimistico["Comune"].unique())
comuni_non_in_df = comuni_hex - comuni_df
print(f"Municipalities in hexagon_df not found in df_2030_ottimistico ({len(comuni_non_in_df)}):")
print(list(comuni_non_in_df))

# Municipalities in df_2030_ottimistico not present in hexagon_df
comuni_non_in_hex = comuni_df - comuni_hex
print(f"Municipalities in df_2030_ottimistico not found in hexagon_df ({len(comuni_non_in_hex)}):")
print(list(comuni_non_in_hex))

Municipalities without hexagons (0):
[]
Municipalities in hexagon_df not found in df_2030_ottimistico (0):
[]
Municipalities in df_2030_ottimistico not found in hexagon_df (0):
[]


#### Visualization of Hexagonal Data with Heatmap Color Coding -  Optimistic Scenario

In [ ]:
import branca.colormap as cm

# Select the column to visualize (you can change it here)
colonna_heatmap = "Domanda Giornaliera (kWh) - Ottimistico per esagono"

# Create the map centered on Brescia
mappa = folium.Map(location=[45.54, 10.22], zoom_start=10)

# Create a continuous color map between min and max
min_val = hexagon_df_ott[colonna_heatmap].min()
max_val = hexagon_df_ott[colonna_heatmap].max()
colormap = cm.linear.YlOrRd_09.scale(min_val, max_val)
colormap.caption = f"{colonna_heatmap}"

# Add hexagons with color based on the variable
for _, row in hexagon_df_ott.iterrows():
    valore = row[colonna_heatmap]
    colore = colormap(valore) if not pd.isna(valore) else "#cccccc"
    folium.Polygon(
        locations=[(y, x) for x, y in row["geometry"].exterior.coords],
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=colore,
        weight=0.2,
        popup=f"{row['Comune']}<br>{colonna_heatmap}: {round(valore, 2)}"
    ).add_to(mappa)

# Add the legend
colormap.add_to(mappa)

# Display the map
mappa

Output hidden; open in https://colab.research.google.com to view.

#### Logarithmic Heatmap Visualization with Detailed Tooltips - Optimistic Scenario

In [ ]:
import numpy as np
import branca.colormap as cm
import folium

# Select the column to visualize for coloring
colonna_heatmap = "Domanda Giornaliera (kWh) - Ottimistico per esagono"

# Create the map centered on Brescia
mappa = folium.Map(location=[45.54, 10.22], zoom_start=10)

# Calculate logarithmic values with offset to avoid log(0)
offset = 1e-6
valori_originali = hexagon_df_ott[colonna_heatmap]
valori_log = np.log10(valori_originali + offset)

# Create a logarithmic colormap
min_log = valori_log.min()
max_log = valori_log.max()
colormap = cm.linear.YlOrRd_09.scale(min_log, max_log)
colormap.caption = f"{colonna_heatmap} (log scale)"

# Add hexagons with detailed tooltips
for _, row in hexagon_df_ott.iterrows():
    valore = row[colonna_heatmap]
    if pd.isna(valore):
        colore = "#cccccc"
    else:
        valore_log = np.log10(valore + offset)
        colore = colormap(valore_log)

    # Create the rich tooltip text
    comune = row['Comune']
    tooltip_text = f"<b>Municipality:</b> {comune.title()}<br>"
    for col in [
        "EV Stimati (2030 - Ottimistico) per esagono",
        "Domanda Giornaliera (kWh) - Ottimistico per esagono",
        "Domanda Mensile (kWh) - Ottimistico per esagono",
        "Domanda Annua (kWh) - Ottimistico per esagono",
        "Colonnine Necessarie - Ottimistico per esagono"
    ]:
        valore_col = row[col]
        valore_mostrato = round(valore_col, 2) if not pd.isna(valore_col) else "N/A"
        tooltip_text += f"<b>{col}:</b> {valore_mostrato}<br>"

    tooltip = folium.Tooltip(tooltip_text, sticky=True)

    folium.Polygon(
        locations=[(y, x) for x, y in row["geometry"].exterior.coords],
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=colore,
        weight=0.2,
        tooltip=tooltip
    ).add_to(mappa)

# Add the legend
colormap.add_to(mappa)

# Display the map
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/02 - Esagoni/log_heatmap_optimistic_scenario.html")

# Save csv

In [ ]:
# Save the current DataFrame (df_comuni) to a CSV file
df_comuni.to_csv("df_attuale_esagoni.csv", index=False)

# Save the 2030 conservative scenario DataFrame (df_2030_conservativo) to a CSV file
df_2030_conservativo.to_csv("df_2030_conservativo_esagoni.csv", index=False)

# Save the 2030 optimistic scenario DataFrame (df_2030_ottimistico) to a CSV file
df_2030_ottimistico.to_csv("df_2030_ottimistico_esagoni.csv", index=False)